In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import glob
%cd ../

/mnt/c/Users/Daniel Yang/Desktop/Final_Project


In [2]:
fileId_path = "experiments/fileIds.pkl"
queryInfo_path = "experiments/queryInfo.pkl"
timeData_path = "experiments/timeData.pkl"
audioFiles_path = "experiments/audioFiles.pkl"
with open(fileId_path, 'rb') as f:
    fileIds=pkl.load(f)
with open(queryInfo_path, 'rb') as f:
    queryInfo=pkl.load(f)
with open(timeData_path, 'rb') as f:
    timeData=pkl.load(f)
with open(audioFiles_path, 'rb') as f:
    audioFiles=pkl.load(f)

In [3]:
def readHypFiles(hypdir):
    data = []
    for hypfile in sorted(glob.glob("experiments/{}/*.hyp".format(hypdir))):
        print(hypfile)
        with open(hypfile, "rb") as f:
            data.append(pkl.load(f))
    return data

In [90]:
def evaluateBeatLevel(tf, segment_predict, solo_name, ref_name, sr=22050):
    # Only evaluate within segments
    diffs = []
    pieceName = solo_name.split('_')[0]
    gt_segments = queryInfo[pieceName]
    annotated_beats = list(timeData[ref_name].keys())
    for idx,beats in enumerate(gt_segments):
        beat1 = beats.split(',')[0][1:]
        beat2 = beats.split(',')[1][:-1]
        idx1 = annotated_beats.index(beat1)
        idx2 = annotated_beats.index(beat2)
        #offset = int(timeData[ref_name][beat1]*sr)-segment_predict[idx][0]
        base_time = tf[segment_predict[idx][0]]
        #base_time = tf[int(timeData[ref_name][beat1]*sr)]
        #print(offset)
        
        new_times = {}
        for i in range(idx1,idx2+1):
            key = annotated_beats[i]
            sample = tf[int(timeData[ref_name][key]*sr)]
            new_time = sample-base_time
            new_times[key]=new_time
            
        gt_beats = list(timeData[solo_name].keys())
        gt_base_time=timeData[solo_name][beat1]
        prev = 0
        prevgt = 0
        for idx, key in enumerate(new_times):
            if key in gt_beats:
                gt_time = (timeData[solo_name][key]-gt_base_time)*sr
                pred_time = new_times[key]
                #f idx != 0:
                #   print("diff",gt_time-prevgt, pred_time-prev)
                prev = pred_time
                prevgt = gt_time
                #rint(gt_time, pred_time)
                diffs.append(np.abs(gt_time-pred_time))
    return diffs

In [91]:
def evaluateBeatLevelFirst(tf, segment_predict, solo_name, ref_name, sr=22050):
    # Only evaluate within segments
    diffs = []
    pieceName = solo_name.split('_')[0]
    gt_segments = queryInfo[pieceName]
    annotated_beats = list(timeData[ref_name].keys())
    for idx,beats in enumerate(gt_segments):
        if idx!=1:
            continue
        beat1 = beats.split(',')[0][1:]
        beat2 = beats.split(',')[1][:-1]
        idx1 = annotated_beats.index(beat1)
        idx2 = annotated_beats.index(beat2)
        offset = int(timeData[ref_name][beat1]*sr)-segment_predict[idx][0]
        base_time = tf[segment_predict[idx][0]]
        #base_time = tf[int(timeData[ref_name][beat1]*sr)]
        #print(offset)
        #print(segment_predict[idx][0])
        #print(int(timeData[ref_name][beat1]*sr))
        
        new_times = {}
        for i in range(idx1,idx2+1):
            key = annotated_beats[i]
            sample = tf[int(timeData[ref_name][key]*sr)]
            new_time = sample-base_time
            new_times[key]=new_time
            
        gt_beats = list(timeData[solo_name].keys())
        gt_base_time=timeData[solo_name][beat1]
        prev = 0
        prevgt = 0
        for idx, key in enumerate(new_times):
            if key in gt_beats:
                gt_time = (timeData[solo_name][key]-gt_base_time)*sr
                pred_time = new_times[key]
                #f idx != 0:
                #   print("diff",gt_time-prevgt, pred_time-prev)
                prev = pred_time
                prevgt = gt_time
                #rint(gt_time, pred_time)
                diffs.append(np.abs(gt_time-pred_time))
    return diffs

In [97]:
def evaluate(hypdir, sr=22050):
    data = readHypFiles(hypdir)
    times = []
    segment_tolerances =[1,.5, 0.25, 0.125, 0.01]
    segment_total = np.zeros(len(segment_tolerances))
    segment_correct = np.zeros(len(segment_tolerances))
    
    beat_tolerances =np.arange(0.05, 2, 0.05)
    #beat_tolerances=[0.25]
    beat_total = np.zeros(len(beat_tolerances))
    beat_correct = np.zeros(len(beat_tolerances))
    beat_totalFirst = np.zeros(len(beat_tolerances))
    beat_correctFirst = np.zeros(len(beat_tolerances))
    for solo_name, ref_name, tf, segment_predict, time_taken in data:
        segment_predict = np.array(segment_predict)*512
        #print(segment_predict)
        segment_diffs, gt_samples, segment_predict = evaluateSegmentLevel(segment_predict, solo_name, ref_name)
        beat_diffs = evaluateBeatLevel(tf, segment_predict, solo_name, ref_name)
        beat_diffsFirst = evaluateBeatLevelFirst(tf, segment_predict, solo_name, ref_name)
        #beat_diffs = evaluateDiffLevel(tf, segment_predict, solo_name, ref_name)
        segment_diffs =np.array(segment_diffs)/sr
        beat_diffs =(np.array(beat_diffs)/sr)
        beat_diffsFirst =(np.array(beat_diffsFirst)/sr)
        times.append(time_taken)
        
        #Update Segment Level Accuracy
        for idx, tolerance in enumerate(segment_tolerances):
            segment_correct[idx] += np.sum(np.where(segment_diffs<tolerance, 1, 0))
            segment_total[idx]+=len(segment_diffs)
            
            
        #Update Beat Level Accuracy
        for idx, tolerance in enumerate(beat_tolerances):
            beat_correct[idx] += np.sum(np.where(beat_diffs<tolerance, 1, 0))
            beat_total[idx]+=len(beat_diffs)
            
        #Update First Beat Level Accuracy
        for idx, tolerance in enumerate(beat_tolerances):
            beat_correctFirst[idx] += np.sum(np.where(beat_diffsFirst<tolerance, 1, 0))
            beat_totalFirst[idx]+=len(beat_diffsFirst)
    print(segment_correct/segment_total)
    print(beat_correct/beat_total)
    #print(beat_correctFirst/beat_totalFirst)
    #print(segment_diffs)
    #print(beat_correct, beat_total)
    #print(gt_samples)
    #print(segment_predict)        

In [98]:
def evaluateSegmentLevel(segment_predict, solo_name, ref_name, sr=22050):
    pieceName = solo_name.split('_')[0]
    gt_segments = queryInfo[pieceName]
    gt_samples = []
    diffs = []
    for beats in gt_segments:
        beat1 = beats.split(',')[0][1:]
        beat2 = beats.split(',')[1][:-1]
        sample1 = timeData[ref_name][beat1]*sr
        sample2 = timeData[ref_name][beat2]*sr
        gt_samples.append((sample1, sample2))
    for idx,segment in enumerate(segment_predict):
        seg0 = segment[0]
        seg1 = segment[1]
        diff0 = np.abs(gt_samples[idx][0]-seg0)
        diff1 = np.abs(gt_samples[idx][1]-seg1)
        diffs.append(diff0)
        diffs.append(diff1)
        #print((gt_samples[idx][0]-seg0)/(sr*sr))
    return diffs, gt_samples, segment_predict

In [99]:
evaluate("debug6")

experiments/debug6/p1s_perf12-p1f_perf16.hyp
experiments/debug6/p1s_perf12-p1f_perf2.hyp
experiments/debug6/p1s_perf12-p1f_perf23.hyp
experiments/debug6/p1s_perf12-p1f_perf24.hyp
experiments/debug6/p1s_perf12-p1f_perf6.hyp
experiments/debug6/p1s_perf2-p1f_perf16.hyp
experiments/debug6/p1s_perf2-p1f_perf2.hyp
experiments/debug6/p1s_perf2-p1f_perf23.hyp
experiments/debug6/p1s_perf2-p1f_perf24.hyp
experiments/debug6/p1s_perf2-p1f_perf6.hyp
experiments/debug6/p1s_perf25-p1f_perf16.hyp
experiments/debug6/p1s_perf25-p1f_perf2.hyp
experiments/debug6/p1s_perf25-p1f_perf23.hyp
experiments/debug6/p1s_perf25-p1f_perf24.hyp
experiments/debug6/p1s_perf25-p1f_perf6.hyp
experiments/debug6/p1s_perf3-p1f_perf16.hyp
experiments/debug6/p1s_perf3-p1f_perf2.hyp
experiments/debug6/p1s_perf3-p1f_perf23.hyp
experiments/debug6/p1s_perf3-p1f_perf24.hyp
experiments/debug6/p1s_perf3-p1f_perf6.hyp
experiments/debug6/p1s_perf9-p1f_perf16.hyp
experiments/debug6/p1s_perf9-p1f_perf2.hyp
experiments/debug6/p1s_perf9-p1

/home/dyang/anaconda3/envs/MIR5/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in less
/home/dyang/anaconda3/envs/MIR5/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in less


[0.85579196 0.82624113 0.67257683 0.5248227  0.06382979]
[0.46520782 0.60645052 0.66541099 0.6989757  0.72107001 0.73914332
 0.75256074 0.76339626 0.7707187  0.77706764 0.78350123 0.78917295
 0.7934479  0.79831542 0.80204013 0.80559553 0.80889698 0.81160586
 0.81431474 0.81698129 0.81913993 0.82167951 0.82375349 0.82582748
 0.82781681 0.82946754 0.83124524 0.83306527 0.83454669 0.83662067
 0.83831372 0.8400491  0.84144586 0.84301194 0.84457801 0.84622873
 0.84766782 0.84948785 0.85075764]


In [81]:
queryInfo['p3s']

['(1.1,12.3)', '(13.1,35.1)', '(35.3,57.3)', '(58.1,71.1)', '(71.3,80.1)']

In [82]:
sr=22050
solo_name, ref_name = 'p3s_perf22', 'p3f_perf23'
int(timeData[ref_name]['12.3']*sr)-int(timeData[ref_name]['1.1']*sr)

636328

In [64]:
int(timeData[solo_name]['24.1']*sr)-int(timeData[solo_name]['9.1']*sr)

622280

In [65]:
queryInfo['p1s']

['(9.1,24.1)', '(29.1,36.1)', '(49.1,68.1)', '(68.1,95.1)', '(96.1,109.1)']

In [66]:
timeData['p1f_perf23']

{'1.1': 0.7393801963333333,
 '1.2': 1.458956916,
 '2.1': 2.19303099,
 '2.2': 2.815238095333333,
 '3.1': 3.461980347333333,
 '3.2': 4.078760393333334,
 '4.1': 4.729070294666666,
 '4.2': 5.400090703,
 '5.1': 6.018654573,
 '5.2': 6.651186697,
 '6.1': 7.249297052333333,
 '6.2': 7.944081632666667,
 '7.1': 8.635631141333333,
 '7.2': 9.349130763666667,
 '8.1': 9.972804233,
 '8.2': 10.94,
 '9.1': 12.544036281333334,
 '9.2': 13.276462585333334,
 '10.1': 14.010113378666667,
 '10.2': 14.742418745333334,
 '11.1': 15.459410430666667,
 '11.2': 16.144323507333336,
 '12.1': 16.866455026666667,
 '12.2': 17.63088435366667,
 '13.1': 18.41077853366667,
 '13.2': 19.130234316,
 '14.1': 19.889236583333332,
 '14.2': 20.651020408,
 '15.1': 21.42299319733333,
 '15.2': 22.121632652999995,
 '16.1': 22.859833711333334,
 '16.2': 23.630113378999997,
 '17.1': 24.394996220666666,
 '17.2': 25.254089191000002,
 '18.1': 26.001254724333336,
 '18.2': 26.728223734,
 '19.1': 27.46341647766667,
 '19.2': 28.170007558333335,
 '